# Image embeddings analysis

In [2]:
import pandas as pd
import numpy as np
import csv
from tqdm import tqdm

### Create mapping id -> category

In [25]:
big_dict = dict()

with open("./Amazon_Sports_and_Outdoors.item", 'r') as f:
    reader = csv.reader(f, delimiter='\t')

    _ = reader.__next__()

    for entry in tqdm(reader, total=1587220):
        big_dict[entry[0]] = entry[4].split(',')[0]

100%|█████████▉| 1587219/1587220 [00:04<00:00, 323382.68it/s]


In [28]:
print(list(big_dict.keys())[:10])
print([entry for entry in list(big_dict.values()) if entry != "'Sports & Outdoors'"])

['884509', '561856', '239749', '55030', '1277121', '497575', '459205', '363349', '342770', '83201']
['', '', '', '', '', '', '', '', '', '', "'Hunting & Fishing'", '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', "'Hunting & Fishing'", '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', "'Sports & Outdoor Recreation Accessories'", '', 

In [ ]:
df1 = pd.read_csv('./Amazon_Sports_and_Outdoors.image', delimiter='\t')

df1.rename(columns={"ent_emb:float_seq": "emb", "ent_id:token": "id"}, inplace=True)

In [ ]:
df = pd.read_csv('./Amazon_Sports_and_Outdoors.image', delimiter='\t')

df.rename(columns={"ent_emb:float_seq": "emb", "ent_id:token": "id"}, inplace=True)
df["emb"] = df["emb"].apply(lambda x: np.fromstring(x, dtype=np.float32, sep=' '))
E = np.vstack(df['emb'].values) # [n, d]

In [ ]:
import scann

searcher = scann.scann_ops_pybind.builder(E, 10, "dot_product").tree(
    num_leaves=1000, num_leaves_to_search=100, training_sample_size=250000
).score_ah(2, anisotropic_quantization_threshold=0.2).reorder(100).build()

In [ ]:
from tqdm import tqdm

y_pred = np.array([])
y_true = np.array([])

for i, emb in tqdm(enumerate(E), total=len(E)):
    dist, ids = searcher.search(emb)

    my_category = df["emb"].iloc[i]
    neigh_categories = df["emb"].iloc[ids]

    print(my_category, neigh_categories)
